In [2]:
# Load silver and scored tables
silver = spark.read.table("silver")
scored = spark.read.table("customer_churn_scored")

# Join on CustomerID
df = silver.join(scored, on="CustomerID", how="inner")

from pyspark.sql.functions import col, when, count, sum, avg

# Create Gold summary table by churn risk segment
gold_summary = (
    df.groupBy("ChurnRiskSegment")
    .agg(
        count("*").alias("TotalCustomers"),
        sum(when(col("ChurnLabel") == "Churned", 1).otherwise(0)).alias("ChurnedCustomers"),
        (sum(when(col("ChurnLabel") == "Churned", 1).otherwise(0)) / count("*")).alias("ChurnRate"),
        avg("MonthlyCharge").alias("AvgMonthlyCharge"),
        avg("ChurnProbability").alias("AvgChurnProbability")
    )
)

display(gold_summary)



StatementMeta(, 5c37c776-b329-42cd-a0bd-1909f3fec398, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, c2f3ab49-f43e-4dda-aaad-a6cca2a0ada6)

In [3]:
# Save as gold table
gold_summary.write.mode("overwrite").saveAsTable("churn_summary_byrisksegment")

StatementMeta(, 5c37c776-b329-42cd-a0bd-1909f3fec398, 5, Finished, Available, Finished)